**Importiere die notwendigen Packages**

In [4]:
import pandas as pd
import pivottablejs
from pivottablejs import pivot_ui
import numpy as np
import altair as alt
import matplotlib.pyplot as plt
from datetime import datetime
import geopandas as gpd
import folium 



Definiere Settings. Hier das Zahlenformat von Float-Werten (z.B. *'{:,.2f}'.format* mit Komma als Tausenderzeichen), 

In [5]:
pd.options.display.float_format = '{:.0f}'.format
pd.set_option('display.width', 100)
pd.set_option('display.max_columns', 15)

Importiere einen Datensatz von der **DWH_INT** Dropzone.

- Beachte dabei die Notation des Pfades...
- Definiere mal aktuell noch keine weiteren Parameter beim Import

**Dataset auf INTEG Datenkatalog**: https://data.integ.stadt-zuerich.ch/dataset/int_dwh_bev_bestand_jahr_herkunft_geschlecht_OD3222

(int_dwh)

In [10]:
# Filepath
fp = r"//szh.loc/ssz/applikationen/OGD_Dropzone/INT_DWH/bev_bestand_jahr_herkunft_geschlecht_OD3222/BEV322OD3222.csv"


# Read the data
data2betested = pd.read_csv(
    fp, 
    sep=',',
   #usecols=[0,1,2,3,4,6],
   #names=['stationid','year','month','avgtemp','latitude','longitude','elevation','station','countryid','country'],
   #skiprows=1,
   # parse_dates=[0],
    parse_dates=True,
    #date_parser=lambda x: pd.to_datetime(x, format="%Y"), 
    #index_col="date_time",
    low_memory=False    
    )


In der Folge ein paar erste Tests:

In [11]:
type(data2betested)

pandas.core.frame.DataFrame

 - 1) Zeige eine kurze Vorschau der importierten Daten
 - 2) Weise die Datentypen aus
 - 3) Zeige die Shape (Umfang) des Datensatzes an

In [12]:
data2betested.head(3)

,StichtagDatJahr,AnzBestWir,AnzBestWirCH,AnzBestWirCHW,AnzBestWirCHM,AnzBestWirAUSL,AnzBestWirAUSLW,AnzBestWirAUSLM,AuslAnt
0,2020,434736,295333,152124,143209,139403,64855,74548,32
1,2019,434008,294205,151730,142475,139803,65125,74678,32
2,2018,428737,290349,149950,140399,138388,64387,74001,32


In [13]:
data2betested.dtypes

StichtagDatJahr      int64
AnzBestWir           int64
AnzBestWirCH         int64
AnzBestWirCHW      float64
AnzBestWirCHM      float64
AnzBestWirAUSL       int64
AnzBestWirAUSLW    float64
AnzBestWirAUSLM    float64
AuslAnt            float64
dtype: object

In [14]:
data2betested.shape

(120, 9)

Beschreibe einzelne Attribute

In [15]:
data2betested.describe()

,StichtagDatJahr,AnzBestWir,AnzBestWirCH,AnzBestWirCHW,AnzBestWirCHM,AnzBestWirAUSL,AnzBestWirAUSLW,AnzBestWirAUSLM,AuslAnt
count,120,120,120,87,87,120,87,87,120
mean,1960,333896,266599,166571,142320,67297,35390,39239,21
std,35,87201,80162,23165,20276,31975,14284,20169,9
min,1901,150301,108291,136041,116422,24306,14326,9980,7
25%,1931,255375,217678,145675,125126,37768,22578,16386,14
50%,1960,361234,278909,161887,138104,66316,32389,38576,19
75%,1990,396840,323283,187400,161720,86396,47712,56700,29
max,2020,440180,386147,207251,178896,139803,65125,74678,34


Wie viele Nullwerte gibt es im Datensatz?

In [16]:
data2betested.isnull().sum()

StichtagDatJahr     0
AnzBestWir          0
AnzBestWirCH        0
AnzBestWirCHW      33
AnzBestWirCHM      33
AnzBestWirAUSL      0
AnzBestWirAUSLW    33
AnzBestWirAUSLM    33
AuslAnt             0
dtype: int64

Wandle die Spalte Jahr von Int zu **Datetime** um. Dies wird in den meisten Grafiken so benötigt.

In [17]:
data2betested['StichtagDatJahr']=pd.to_datetime(data2betested['StichtagDatJahr'], format='%Y')

#TO DO --> setze das Datum auf den 31.12. der Jahres.
#test=data2betested[['StichtagDatJahr']].date().replace(month=12, day=31)

data2betested

,StichtagDatJahr,AnzBestWir,AnzBestWirCH,AnzBestWirCHW,AnzBestWirCHM,AnzBestWirAUSL,AnzBestWirAUSLW,AnzBestWirAUSLM,AuslAnt
0,2020-01-01,434736,295333,152124,143209,139403,64855,74548,32
1,2019-01-01,434008,294205,151730,142475,139803,65125,74678,32
2,2018-01-01,428737,290349,149950,140399,138388,64387,74001,32
3,2017-01-01,423310,286255,148151,138104,137055,63790,73265,32
4,2016-01-01,415682,282209,146356,135853,133473,62134,71339,32
...,...,...,...,...,...,...,...,...,...
115,1905-01-01,168877,117236,nan,nan,51641,nan,nan,31
116,1904-01-01,164060,115612,nan,nan,48448,nan,nan,30
117,1903-01-01,158719,112915,nan,nan,45804,nan,nan,29
118,1902-01-01,153741,110707,nan,nan,43034,nan,nan,28


Bereite die Daten für die Grafik unten vor

In [18]:
Jahre = data2betested.groupby("StichtagDatJahr")
len(Jahre)

120

In [19]:
for key, group in Jahre:
    print(key, len(group))

1901-01-01 00:00:00 1
1902-01-01 00:00:00 1
1903-01-01 00:00:00 1
1904-01-01 00:00:00 1
1905-01-01 00:00:00 1
1906-01-01 00:00:00 1
1907-01-01 00:00:00 1
1908-01-01 00:00:00 1
1909-01-01 00:00:00 1
1910-01-01 00:00:00 1
1911-01-01 00:00:00 1
1912-01-01 00:00:00 1
1913-01-01 00:00:00 1
1914-01-01 00:00:00 1
1915-01-01 00:00:00 1
1916-01-01 00:00:00 1
1917-01-01 00:00:00 1
1918-01-01 00:00:00 1
1919-01-01 00:00:00 1
1920-01-01 00:00:00 1
1921-01-01 00:00:00 1
1922-01-01 00:00:00 1
1923-01-01 00:00:00 1
1924-01-01 00:00:00 1
1925-01-01 00:00:00 1
1926-01-01 00:00:00 1
1927-01-01 00:00:00 1
1928-01-01 00:00:00 1
1929-01-01 00:00:00 1
1930-01-01 00:00:00 1
1931-01-01 00:00:00 1
1932-01-01 00:00:00 1
1933-01-01 00:00:00 1
1934-01-01 00:00:00 1
1935-01-01 00:00:00 1
1936-01-01 00:00:00 1
1937-01-01 00:00:00 1
1938-01-01 00:00:00 1
1939-01-01 00:00:00 1
1940-01-01 00:00:00 1
1941-01-01 00:00:00 1
1942-01-01 00:00:00 1
1943-01-01 00:00:00 1
1944-01-01 00:00:00 1
1945-01-01 00:00:00 1
1946-01-01

In [20]:
myJahr = "2020-01-01"
aktuellesJahr=Jahre.get_group(myJahr)   
aktuellesJahr.head(3)

,StichtagDatJahr,AnzBestWir,AnzBestWirCH,AnzBestWirCHW,AnzBestWirCHM,AnzBestWirAUSL,AnzBestWirAUSLW,AnzBestWirAUSLM,AuslAnt
0,2020-01-01,434736,295333,152124,143209,139403,64855,74548,32


**Selektionen und Aggregationen**

Check this [Doku](https://www.shanelynn.ie/summarising-aggregation-and-grouping-data-in-python-pandas/)

In [22]:
#test = data2betested.groupby(['StichtagDatJahr', 'AlterV20Kurz'])[['AnzBestWir']].agg()

# produces Pandas Series
    #data2betested.groupby(['StichtagDatJahr', 'KonOGDSort'])['AnzBestWir'].sum()
# Produces Pandas DataFrame
    #data2betested.groupby(['StichtagDatJahr', 'KonOGDSort'])[['AnzBestWir']].sum()

    

**Einfache Visualisierungen zur Plausi**

Exploriere die Daten mit Pivottable.JS

In [23]:
from pivottablejs import pivot_ui

pivot_ui(data2betested)

Liniendiagramm 
[Link zur Doku](https://altair-viz.github.io/gallery/multiline_highlight.html)

In [25]:
mySelection = data2betested[(data2betested.StichtagDatJahr>"1900-01-01")]
#mySelection = data2betested[(data2betested.StichtagDatJahr>"2011-01-01") & (data2betested.QuarSort<70)]
mySelection.head()



,StichtagDatJahr,AnzBestWir,AnzBestWirCH,AnzBestWirCHW,AnzBestWirCHM,AnzBestWirAUSL,AnzBestWirAUSLW,AnzBestWirAUSLM,AuslAnt
0,2020-01-01,434736,295333,152124,143209,139403,64855,74548,32
1,2019-01-01,434008,294205,151730,142475,139803,65125,74678,32
2,2018-01-01,428737,290349,149950,140399,138388,64387,74001,32
3,2017-01-01,423310,286255,148151,138104,137055,63790,73265,32
4,2016-01-01,415682,282209,146356,135853,133473,62134,71339,32


In [26]:
#mySelection = aktuellesJahr[["StatZoneSort","AnzBestWir"]]
#mySelection

In [32]:
# df_statzonen = df_statzonen[(df_statzonen.knr>0) & (df_statzonen.qname!="Hirslanden")]
#aktuellesJahr[[aktuellesJahr.KreisSort =7]])
highlight = alt.selection(type='single', on='mouseover',
                          fields=['AnzBestWirAUSL'], nearest=True)
#x='date:StichtagDatJahr',
base = alt.Chart(mySelection).encode(
    x='StichtagDatJahr:N',
    y='AnzBestWirAUSL',
    color='AnzBestWirAUSL'
)

points = base.mark_circle().encode(
    opacity=alt.value(0)
).add_selection(
    highlight
).properties(
    width=700 , height=300
)
lines = base.mark_line().encode(
    size=alt.condition(~highlight, alt.value(1.5), alt.value(4))
).interactive()

points + lines

alt.LayerChart(...)

In [39]:
alt.Chart(data2betested).mark_circle(size=60).encode(
    x='AnzBestWir',
    y='StichtagDatJahr',
    #color='AnzNat',
    #color=alt.Color('AnzNat', scale=alt.Scale(scheme='dark2')),
    color=alt.Color('AnzBestWirAUSL', scale=alt.Scale(scheme='greens')),
    tooltip=['AnzBestWir','StichtagDatJahr','AnzBestWirAUSL']
).interactive() # this makes the axes interactive: now you can zoom & pan




alt.Chart(...)

**Sharepoint als gecheckt markieren!**

Record auf Sharepoint: **[Link](http://kollaboration.intranet.stzh.ch/orga/ssz-produkte/Lists/SASA_Outputs/EditForm.aspx?ID=41&Source=%2Forga%2Fssz%2Dprodukte%2FLists%2FSASA%5FOutputs)**